In [13]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [25]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))
  
  list_for_df = []
  
  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]
   
    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          "X-RapidAPI-Key": "0dc165bb4fmsh6e66l9b050bma2dp1fb4afjsnfb0f7c4c9dc9",
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['arrival_icao'] = icao
        flights_dict['arrival_time'] = flight['arrival'].get('scheduledTimeLocal', None)
        
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)
 

In [29]:
icaos = ['LOWI'] 
 #['EDDB','EDDT','EDDH','EDDM','EDDS','EDJA','EGMC','EGSH','LEBL','LEGE','LERS','LOWI']

In [30]:
result= tomorrows_flight_arrivals(icaos)
flight_data=pd.DataFrame(result)

In [31]:
flight_data

,flight_number,departure_icao,arrival_icao,arrival_time
0,OS 903,LOWW,LOWI,2022-10-13 10:30+02:00
1,OS 901,LOWW,LOWI,2022-10-13 18:10+02:00
2,U2 8291,EGKK,LOWI,2022-10-13 15:30+02:00
3,EN 8052,EDDF,LOWI,2022-10-13 13:50+02:00
4,OS 913,LOWW,LOWI,2022-10-13 22:50+02:00
5,EN 8056,EDDF,LOWI,2022-10-13 22:10+02:00


In [32]:
flight_data['arrival_time']=pd.to_datetime(flight_data['arrival_time']).dt.tz_convert(None)
flight_data

,flight_number,departure_icao,arrival_icao,arrival_time
0,OS 903,LOWW,LOWI,2022-10-13 08:30:00
1,OS 901,LOWW,LOWI,2022-10-13 16:10:00
2,U2 8291,EGKK,LOWI,2022-10-13 13:30:00
3,EN 8052,EDDF,LOWI,2022-10-13 11:50:00
4,OS 913,LOWW,LOWI,2022-10-13 20:50:00
5,EN 8056,EDDF,LOWI,2022-10-13 20:10:00


In [220]:
schema="gans_database"   
host="127.0.0.1"        
user="root"
password="my_module.mypassword" 
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'       

In [221]:
flights.to_sql('flights',        
              if_exists='append', 
              con=con,           
              index=False)

4